In [1]:
from __future__ import print_function

import sys
import socket
import os

repo_path = os.path.expanduser('~/repo/seg_uncertainty')
sys.path.extend([repo_path])

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm

from datetime import datetime
from pytz import timezone
import copy

import numpy as np
import cv2
import SimpleITK as sitk
import torch
from tqdm import tqdm

import dill
import glob
from collections import OrderedDict

from datasets.data_config import get_config

%matplotlib inline
%load_ext autoreload
%autoreload 2
print("Ready")

Ready


## Generate latex table entries

- Don't forget to adjust `model_name`, `mc_dropout`, `src_path`
- **Parameter** `result_type`: **dt_base**, baseline for the combined segmentation and detection task. **detection**, actual achieved results through combined approach

In [9]:
from evaluate.test_results import load_results
from evaluate.test_results import load_results_sim_correction

model_name = "unet_mc_dice"

for mc_dropout in [False, True]:
    print("================ {} =============== {} ============".format(model_name, mc_dropout))
    src_path = "~/expers/redo_expers/*/" + model_name
    res_seg_only, latex_dsc, latex_hd = load_results(src_path, mc_dropout=mc_dropout, num_fold=4, 
                                                     file_prefix="results", num_patients=25)

    src_path = "~/expers/redo_expers/" + model_name


    _, latex_dsc_dt_base, latex_hd_dt_base = load_results_sim_correction(
                                                src_path, mc_dropout=mc_dropout, 
                                             result_type="det_base", base_results=res_seg_only)

    _, latex_dsc_detect, latex_hd_detect = load_results_sim_correction(
                                                src_path, mc_dropout=mc_dropout, 
                                             result_type="detection", base_results=res_seg_only)

    print("-------------------------------- {} {} ----------------------------".format(model_name,
                                                                                   mc_dropout))
    print(latex_dsc)
    print(latex_dsc_dt_base)
    print(latex_dsc_detect)
    print("-------------------------------- {} ----------------------------".format("Hausdorff"))
    print(latex_hd)
    print(latex_hd_dt_base)
    print(latex_hd_detect)


================ unet_mc_dice =============== False ============
INFO - /home/jorg/expers/redo_expers/f0/unet_mc_dice/results_f0_25.npz
INFO - /home/jorg/expers/redo_expers/f1/unet_mc_dice/results_f1_25.npz
INFO - /home/jorg/expers/redo_expers/f2/unet_mc_dice/results_f2_25.npz
INFO - /home/jorg/expers/redo_expers/f3/unet_mc_dice/results_f3_25.npz
Load results from /home/jorg/expers/redo_expers/unet_mc_dice/results_fall_corrall_100.npz
Load results from /home/jorg/expers/redo_expers/unet_mc_dice/results_fall_100.npz
-------------------------------- unet_mc_dice False ----------------------------
 & U-net-SD &  &  & 0.965$\pm$0.02 & 0.928$\pm$0.04 & 0.888$\pm$0.03  & 0.914$\pm$0.08 & 0.844$\pm$0.09 & 0.896$\pm$0.03 \\ 
 &  & \textbf{x} &  & *0.970$\pm$0.01 & *0.961$\pm$0.01 & *0.904$\pm$0.03  & *0.949$\pm$0.03 & *0.938$\pm$0.03 & *0.924$\pm$0.02 \\ 
 &  &  & \textbf{x} & 0.968$\pm$0.01 & *0.942$\pm$0.03 & 0.897$\pm$0.03  & 0.928$\pm$0.06 & *0.881$\pm$0.07 & *0.909$\pm$0.03 \\ 
----------